Taller Final Gradient Boosting 

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import time
import shap  # for model interpretability
import os

# Check the current working directory to confirm the dataset's presence
print("Current Working Directory: ", os.getcwd())

# Load and display the first few rows of the dataset
file_path = 'creditcard_2023.csv'  # assuming the file is in the current working directory
data = pd.read_csv(file_path)
print(data.head())

# Check for duplicates
print("\nNumber of duplicate rows:", data.duplicated().sum())

# Remove duplicates
data = data.drop_duplicates()

# Check the shape of the data after removing duplicates
print("\nData shape after removing duplicates:", data.shape)

# Check for any missing values in the dataset
print("\nMissing values in each column:\n", data.isnull().sum())
